## Imports

In [1]:
import numpy as np
import open3d as o3d
import trimesh

## Load File

This loads a point cloud. This cloud is already stitched together and all outliers have been removed.

In [2]:
point_cloud = o3d.io.read_point_cloud('../data/complete_cluster_3.pcd')
#The point cloud should have normals, but just in case we create the here
point_cloud.estimate_normals()

# Step by Step

## Meshing

We tried the different meshing algorithms of open3d to find the best one for this purpose.

#### Ball Pivot

We found that ball pivot meshing gives really bad results. WWe included it here nonetheless so you can check for yourself.  However we didn't consider it any further after that.

In [3]:
radii = [0.02, 0.04, 0.1, 0.2, 0.5]
bp_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(point_cloud, o3d.utility.DoubleVector(radii))

In [4]:
o3d.visualization.draw_geometries([bp_mesh], mesh_show_back_face=True)

#### Poisson Meshing

Poisson Meshing looked a lot more promising. It closes a lot of holes successfully and it is really fast. However it also introduces a lot more artifacts which we could not get rid of. Those are caused by holes in the scan. If all the holes are closed it might be worth to reconsider this approach. In the end we decided that it was not worth it. Especially because we are not sure if patching the holes is really the way to go, because that would mean that we have to assume some information, which is not ideal for medical scans.

In [5]:
pm_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(point_cloud, depth=10)
#The mesh has no normals yet, so we have to create them
pm_mesh.compute_vertex_normals()

TriangleMesh with 1025601 points and 2052723 triangles.

In [6]:
o3d.visualization.draw_geometries([pm_mesh], mesh_show_back_face=True)

#### Alpha Shape Meshing

This algorithm gave us the best results. There are holes in the mesh, but as already mentioned we don't think patching them without getting additional data from scans is a good idea. The alpha meshing algorithm only needs one parameter which is called alpha. Alpha determines the largest size an edge between to points can be. If alpha is to small then some points that should be connected won't be, which leads to holes in the mesh. If alpha is to large then points that shouldn't be connected will be, which removes detail from the scan. We would that the best compromise is an alpha of 0.2.



Open3d's explenation of the algorithm:

Alpha shapes
The alpha shape [Edelsbrunner1983] is a generalization of a convex hull. As described here [https://graphics.stanford.edu/courses/cs268-11-spring/handouts/AlphaShapes/as_fisher.pdf] one can intuitively think of an alpha shape as the following: Imagine a huge mass of ice cream containing the points S as hard chocolate pieces. Using one of these sphere-formed ice cream spoons we carve out all parts of the ice cream block we can reach without bumping into chocolate pieces, thereby even carving out holes in the inside (e.g., parts not reachable by simply moving the spoon from the outside). We will eventually end up with a (not necessarily convex) object bounded by caps, arcs and points. If we now straighten all round faces to triangles and line segments, we have an intuitive description of what is called the alpha shape of S.

In [7]:
#Example at alpha = 0.1 . This has to many small holes.
as_mesh1 = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, 0.1)  
as_mesh1.compute_vertex_normals()   

o3d.visualization.draw_geometries([as_mesh1], mesh_show_back_face=True)

In [8]:
#Example at alpha = 0.2 . This is what we determined as ideal compromise.
as_mesh2 = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, 0.2)  
as_mesh2.compute_vertex_normals()  

o3d.visualization.draw_geometries([as_mesh2], mesh_show_back_face=True)

In [9]:
#Example at alpha = 0.5 . This removes to much detail.
as_mesh5 = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, 0.5)  
as_mesh5.compute_vertex_normals() 

o3d.visualization.draw_geometries([as_mesh5], mesh_show_back_face=True)

One problem with open3d's alpha meshing is that some of the faces are oriented the wrong way. If you press Ctrl+9 in the visualizer you can see that the normals are all over the place:

In [10]:
o3d.visualization.draw_geometries([as_mesh2], mesh_show_back_face=True)

Fixing the triangle orientation is a bit time consuming and is  not always necessary, but it might save a lot of trouble later. Another package called 'trimesh' can do the fixing for us. However we have to convert the mesh first. 

In [11]:
#chose which mesh to fix:
mesh = as_mesh2

#convert to trimesh
tmesh = trimesh.Trimesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles), vertex_normals=np.asarray(mesh.vertex_normals))

#fix normals
trimesh.repair.fix_normals(tmesh)

#convert back to open3d mesh. We also have to calculate the new normals after convertion
fixed_mesh = tmesh.as_open3d
fixed_mesh.compute_vertex_normals()

TriangleMesh with 197516 points and 409036 triangles.

In [12]:
o3d.visualization.draw_geometries([fixed_mesh], mesh_show_back_face=True)

## Smoothing

The mesh is still very rough. Presumably due to noise from the scan. A simple smoothing algorithm can solve that. It should be said here that we did not do any evaluation on the correctness of the resulting mesh. This means that we do not know wether or not the meshing and or smoothing removed any detail or changed the form of the object in any undesireable way. We also didn't experiment with the parameters.

In [13]:
smooth_mesh = fixed_mesh.filter_smooth_laplacian(10, 0.5)

#The normals have to be recalculated for every time the faces change
smooth_mesh.compute_vertex_normals()

TriangleMesh with 197516 points and 409036 triangles.

In [14]:
o3d.visualization.draw_geometries([smooth_mesh], mesh_show_back_face=True)

# Functions

The following code does the same, it is just splitt into functions for easier exploration.

In [15]:
def make_mesh(cloud, alpha = 0.2):
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(cloud, alpha)       
    mesh.compute_vertex_normals()    
    tmesh = trimesh.Trimesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles), vertex_normals=np.asarray(mesh.vertex_normals))
    trimesh.repair.fix_normals(tmesh)
    return tmesh.as_open3d
    
def smooth(mesh,  smooth_iterations = 10, smooth_lamb = 0.5):
    smooth_mesh = fixed_mesh.filter_smooth_laplacian(smooth_iterations, smooth_lamb)
    smooth_mesh.compute_vertex_normals()
    return smooth_mesh
    
def make_smooth_mesh(cloud, alpha = 0.2, smooth_iterations = 10, smooth_lamb = 0.5):    
    return smooth(make_mesh(cloud, alpha), smooth_iterations, smooth_lamb)

In [16]:
final_mesh = make_smooth_mesh(point_cloud)  #optional parameters: (point_cloud, alpha = 0.2, smooth_iterations = 10, smooth_lamb = 0.5)
o3d.visualization.draw_geometries([final_mesh], mesh_show_back_face=True)